# Capstone Project

## Modeling Notebook - `PREPOST_dataset_COMBO` from 11/14/19

#### Importing Libraries

In [1]:
%matplotlib inline

# general libraries
import re
import string
import sys
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# importing date libraries
import datetime as dt
import dateutil.parser as dparser

# scikit-learn libraries for preprocessing
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer

# scikit-learn libraries for constructing pipelines
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.base import clone

# scikit-learn libraries for clustering and dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.mixture import GaussianMixture

# scikit-learn libraries for evaluation
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

# scikit-learn libraries for feature selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import RFECV

# scikit-learn libraries for learning
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_validate, cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# saving models
import pickle
from sklearn.externals import joblib

# setting pandas display options
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 10000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('precision', 5)
pd.options.mode.chained_assignment = None

/anaconda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


#### Directory/File Structure

In [2]:
sys.version

'3.6.9 |Anaconda custom (x86_64)| (default, Jul 30 2019, 13:42:17) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'

In [3]:
print ('Running pandas version:', pd.__version__)
print ('Running numpy version:', np.__version__)
print ('Running sklearn version:', sklearn.__version__)

Running pandas version: 0.25.2
Running numpy version: 1.17.2
Running sklearn version: 0.21.3


In [4]:
os.getcwd()

'/Users/Shailesh/Desktop/MIDS/Capstone/w210CapstoneProject/notebooks'

In [5]:
os.chdir('../data')

In [6]:
sorted(os.listdir())

['.DS_Store',
 'Capstone_Fall_Shannon_Sept2019_request.csv',
 'POSTOP_ALL_col_names_11_9.pkl',
 'POSTOP_TREE_ALL_col_names_11_9.pkl',
 'POSTOP_categorical_TREE_col_names_11_14.pkl',
 'POSTOP_categorical_TREE_col_names_11_9.pkl',
 'POSTOP_categorical_col_names_11_14.pkl',
 'POSTOP_categorical_col_names_11_9.pkl',
 'POSTOP_numerical_col_names_11_14.pkl',
 'POSTOP_numerical_col_names_11_9.pkl',
 'PREOP_ALL_col_names_11_9.pkl',
 'PREOP_TREE_ALL_col_names_11_9.pkl',
 'PREOP_categorical_TREE_col_names_11_14.pkl',
 'PREOP_categorical_TREE_col_names_11_9.pkl',
 'PREOP_categorical_col_names_11_14.pkl',
 'PREOP_categorical_col_names_11_9.pkl',
 'PREOP_dataset_10_24.pkl',
 'PREOP_dataset_TREE_10_24.pkl',
 'PREOP_dataset_TREE_cabg.pkl',
 'PREOP_dataset_TREE_cabvalve.pkl',
 'PREOP_dataset_TREE_valve.pkl',
 'PREOP_dataset_cabg.pkl',
 'PREOP_dataset_cabvalve.pkl',
 'PREOP_dataset_valve.pkl',
 'PREOP_numerical_col_names_11_14.pkl',
 'PREOP_numerical_col_names_11_9.pkl',
 'PREPOST_dataset_COMBO_11_14.p

## Preprocessing Unprocessed Dataset

### Loading Dataset

#### `PREPOST_dataset_COMBO_11_14`

In [7]:
data = pd.read_pickle('../data/PREPOST_dataset_COMBO_11_14.pkl')

In [8]:
data.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,recordId,predstro,cnstrokp,cnstrokttia,cncomaenceph,strokeBin,strokeBin2,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,mtcause_Cardiac,mtcause_Infection,mtcause_Neuro,mtcause_Pulmonary,mtcause_Renal,mtcause_Vascular,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,mtcause,unplproc,urgntrsn
0,54,180.0,117.0,36.11111,43.0,0.9,3.8,7.2,6.5,47.0,42.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,7,4,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,1.0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,3,3,0,4,2,0,0,0,0,0,0,0,0,0

In [9]:
data.shape

(42740, 262)

In [10]:
data= pd.read_csv('../dataWithProcedureType.csv')

In [11]:
data.shape

(42740, 263)

In [12]:
data.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,recordId,predstro,cnstrokp,cnstrokttia,cncomaenceph,strokeBin,strokeBin2,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,mtcause_Cardiac,mtcause_Infection,mtcause_Neuro,mtcause_Pulmonary,mtcause_Renal,mtcause_Vascular,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,mtcause,unplproc,urgntrsn,procTyp
0,54,180.0,117.0,36.11111,43.0,0.9,3.8,7.2,6.5,47.0,42.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,7,4,1,1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,3,3,0,4,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0.014,2,0,2,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,28.0,29.0,150.0,1.2,NaN,NaN,1

In [13]:
cabg = data.loc[data['procTyp'] == 1]
valve = data.loc[data['procTyp'] == 2]
cabgValve = data.loc[data['procTyp'] == 3]

In [14]:
cabg.shape

(28945, 263)

In [15]:
valve.shape

(8500, 263)

In [16]:
cabgValve.shape

(5295, 263)

In [17]:
origData = data

In [18]:
data = cabgValve

### Dropping Irrelevant Columns

- `recordId`
- `cnstrokp`
- `cnstrokttia`
- `cncomaenceph`
- `strokeBin` as we are going to use the more inclusive `strokeBin2` as our outcome variable

#### Relative to prior versions, we are also going to retain `predstro` that will be used to compare our models versus the `STS` Model

In [19]:
cols_to_drop = ['recordId',
                'cnstrokp',
                'cnstrokttia',
                'cncomaenceph',
                'strokeBin',
               'procTyp']

In [20]:
len(cols_to_drop)

6

- dropping columns

In [21]:
data = data.drop(cols_to_drop, axis=1)

In [22]:
data.shape

(5295, 257)

- resetting `DataFrame` `index` 

In [23]:
data = data.reset_index(drop=True)

In [24]:
data.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,predstro,strokeBin2,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,mtcause_Cardiac,mtcause_Infection,mtcause_Neuro,mtcause_Pulmonary,mtcause_Renal,mtcause_Vascular,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,mtcause,unplproc,urgntrsn
0,54,180.00000,117.0,36.11111,43.0,0.9,3.8,7.2,6.5,47.0,42.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,7,4,1,1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,3,3,0,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0.014,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,28.0,29.0,150.0,1.2,NaN,NaN,108.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,

In [25]:
data.tail()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,predstro,strokeBin2,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,mtcause_Cardiac,mtcause_Infection,mtcause_Neuro,mtcause_Pulmonary,mtcause_Renal,mtcause_Vascular,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,mtcause,unplproc,urgntrsn
5290,62,165.10001,77.6,28.46869,31.6,1.55,3.9,5.6,10.61,55.0,NaN,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,9,4,3,1,1,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.026,0,NaN,0,0,NaN,0.0,0.0,1.0,2.0,22.0,31.0,122.0,1.2,NaN,NaN,106.0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,

In [26]:
data.shape

(5295, 257)

### Feature Matrix `X`
- in the next cell you can select which outcome vector you want to use: `strokeBin` or `strokeBin2`

In [27]:
X = data.copy().drop('strokeBin2', axis=1)

In [28]:
X = X.reset_index(drop=True)

In [29]:
X.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,predstro,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,mtcause_Cardiac,mtcause_Infection,mtcause_Neuro,mtcause_Pulmonary,mtcause_Renal,mtcause_Vascular,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,mtcause,unplproc,urgntrsn
0,54,180.0,117.0,36.11111,43.0,0.9,3.8,7.2,6.5,47.0,42.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,7,4,1,1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,3,3,0,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0.014,NaN,0,0,NaN,NaN,NaN,NaN,NaN,28.0,29.0,150.0,1.2,NaN,NaN,108.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,0,0,1,1,0,1,0,0

In [30]:
X.shape

(5295, 256)

### Target Vector `y`

In [31]:
y = data.copy()['strokeBin2']

In [32]:
type(y)

pandas.core.series.Series

In [33]:
y.head(2)

0    0
1    0
Name: strokeBin2, dtype: int64

In [34]:
y.shape

(5295,)

In [35]:
y.unique()

array([0, 1])

In [36]:
y.value_counts()

0    5136
1     159
Name: strokeBin2, dtype: int64

### `X_train_all`, `X_devtest`, `y_train`, `y_devtest`
- using `train_test_split` with `stratify` parameter to ensure relative proportion of outcome classes are the same in `train`, `dev` and `test` sets
- observation split will be `80/10/10` between `train`, `dev`, `test`
- designation of `_all` denotes complete feature set (`PRE` + `POST`)

In [37]:
X_train_all, X_devtest_all, y_train, y_devtest = train_test_split(X,
                                                                  y,
                                                                  test_size=0.2,
                                                                  random_state=0,
                                                                  stratify=y)

#### validating `train_test_split`

In [38]:
X.shape

(5295, 256)

In [39]:
np.rint(X.shape[0] * 0.20)

1059.0

In [40]:
X_train_all.shape, X_devtest_all.shape, y_train.shape, y_devtest.shape

((4236, 256), (1059, 256), (4236,), (1059,))

In [41]:
X.shape[0] - X_train_all.shape[0] - X_devtest_all.shape[0]

0

In [42]:
y.shape[0] - y_train.shape[0] - y_devtest.shape[0]

0

#### validating `stratify` worked
- relative proportion of classes in `y`

In [43]:
print (np.round(y.value_counts()[0] / y.shape[0], 4))
print (np.round(y.value_counts()[1] / y.shape[0], 4))

0.97
0.03


- relative proportion of classes in `y_train`

In [44]:
print (np.round(y_train.value_counts()[0] / y_train.shape[0], 4))
print (np.round(y_train.value_counts()[1] / y_train.shape[0], 4))

0.97
0.03


- relative proportion of classes in `y_devtest`

In [45]:
print (np.round(y_devtest.value_counts()[0] / y_devtest.shape[0], 4))
print (np.round(y_devtest.value_counts()[1] / y_devtest.shape[0], 4))

0.9698
0.0302


### `X_dev_all`, `X_test_all`, `y_dev`, `y_test`

In [46]:
X_dev_all, X_test_all, y_dev, y_test = train_test_split(X_devtest_all,
                                                        y_devtest,
                                                        test_size=0.5,
                                                        random_state=0,
                                                        stratify=y_devtest)

- validating `train_test_split`

In [47]:
X_devtest_all.shape

(1059, 256)

In [48]:
np.rint(X_devtest_all.shape[0] * 0.50)

530.0

In [49]:
X_dev_all.shape, X_test_all.shape, y_dev.shape, y_dev.shape

((529, 256), (530, 256), (529,), (529,))

#### validating `stratify` worked
- relative proportion of classes in `y`

In [50]:
print (np.round(y.value_counts()[0] / y.shape[0], 4))
print (np.round(y.value_counts()[1] / y.shape[0], 4))

0.97
0.03


- relative proportion of classes in `y_devtest`

In [51]:
print (np.round(y_devtest.value_counts()[0] / y_devtest.shape[0], 4))
print (np.round(y_devtest.value_counts()[1] / y_devtest.shape[0], 4))

0.9698
0.0302


- relative proportion of classes in `y_dev`

In [52]:
print (np.round(y_dev.value_counts()[0] / y_dev.shape[0], 4))
print (np.round(y_dev.value_counts()[1] / y_dev.shape[0], 4))

0.9698
0.0302


- relative proportion of classes in `y_test`

In [53]:
print (np.round(y_test.value_counts()[0] / y_test.shape[0], 4))
print (np.round(y_test.value_counts()[1] / y_test.shape[0], 4))

0.9698
0.0302


### Resetting Indicies

In [54]:
X_train_all = X_train_all.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

In [55]:
X_dev_all = X_dev_all.reset_index(drop=True)
y_dev = y_dev.reset_index(drop=True)

In [56]:
X_test_all = X_test_all.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

### Extracting `predstro` from `X_train`, `X_dev` and `X_test`
- and subsequently creating `predstro_train`, `predstro_dev`, `predstro_test`

- `train`

In [57]:
X_train_all.shape

(4236, 256)

In [58]:
predstro_train = X_train_all['predstro']

In [59]:
predstro_train.shape

(4236,)

In [60]:
X_train_all = X_train_all.drop('predstro', axis=1)

In [61]:
X_train_all.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,mtcause_Cardiac,mtcause_Infection,mtcause_Neuro,mtcause_Pulmonary,mtcause_Renal,mtcause_Vascular,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,mtcause,unplproc,urgntrsn
0,80,168.0,119.0,42.1627,41.0,2.00,3.9,NaN,13.10,55.0,19.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,12,2,3,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,3,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,25.8,33.6,141.0,1.8,NaN,NaN,113.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0

In [62]:
X_train_all.shape

(4236, 255)

- `dev`

In [63]:
X_dev_all.shape

(529, 256)

In [64]:
predstro_dev = X_dev_all['predstro']

In [65]:
predstro_dev.shape

(529,)

In [66]:
X_dev_all = X_dev_all.drop('predstro', axis=1)

In [67]:
X_dev_all.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,mtcause_Cardiac,mtcause_Infection,mtcause_Neuro,mtcause_Pulmonary,mtcause_Renal,mtcause_Vascular,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,mtcause,unplproc,urgntrsn
0,85,178.00000,87.0,27.45865,40.8,0.67,3.5,5.7,7.83,65.0,46.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,7,4,3,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,3,3,1,1,2,0,1,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,0.0,2.0,1.0,0.0,26.0,35.1,193.0,1.1,NaN,NaN,77.0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,1,0,1,0,1,0,0,0,1,0,0,0,0,0,

In [68]:
X_dev_all.shape

(529, 255)

- `test`

In [69]:
X_test_all.shape

(530, 256)

In [70]:
predstro_test = X_test_all['predstro']

In [71]:
predstro_test.shape

(530,)

In [72]:
X_test_all = X_test_all.drop('predstro', axis=1)

In [73]:
X_test_all.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,mtcause_Cardiac,mtcause_Infection,mtcause_Neuro,mtcause_Pulmonary,mtcause_Renal,mtcause_Vascular,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,mtcause,unplproc,urgntrsn
0,82,185.0,91.0,26.58875,42.0,1.1,3.7,6.2,7.3,55.0,40.0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,3,0,3,1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,25.0,33.0,175.0,1.3,NaN,NaN,155.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0

In [74]:
X_test_all.shape

(530, 255)

- validating row count - `42,740` total observations

In [75]:
42740 - X_train_all.shape[0] - X_dev_all.shape[0] - X_test_all.shape[0]

37445

#### Last look at the data (`X_train_all`) before modeling

In [76]:
X_train_all.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,mtcause_Cardiac,mtcause_Infection,mtcause_Neuro,mtcause_Pulmonary,mtcause_Renal,mtcause_Vascular,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,mtcause,unplproc,urgntrsn
0,80,168.00000,119.0,42.16270,41.0,2.00,3.9,NaN,13.10,55.0,19.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,12,2,3,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,3,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,25.8,33.6,141.0,1.8,NaN,NaN,113.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,1,0,1,0,1,0,0,0,0,0,

In [77]:
X_train_all.tail()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,mtcause_Cardiac,mtcause_Infection,mtcause_Neuro,mtcause_Pulmonary,mtcause_Renal,mtcause_Vascular,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,mtcause,unplproc,urgntrsn
4231,74,165.10001,78.8,28.90893,31.0,1.1,4.0,6.7,8.2,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,12,3,3,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,3,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,0.0,0.0,0.0,1.0,19.0,34.0,202.0,2.2,NaN,NaN,176.0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,1,0,1,0,1,0,1,0,0,0,0,0,0,0

## Pre-processing Data using `Scikit-Learn` `Pipelines`

### Creating a `FeatureSelector` transformer
- necessary because we are working with heterogeneous data (numerical and categorical features) -- want to be able to pick and choose which features (columns) to pass through our pipelines (and transform them) instead of having to pass through the whole dataframe

In [78]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    '''
    Transformer to select column from a data frame to perform additional transformations on
    Use this for selecting column(s) that require fit transform
    '''
    
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[data_dict.columns.intersection(self.key)]

## Loading Column Names for `PREOP` and `POSTOP` Features
- these `lists` of features will be used to mask/select specified features from the `COMBO` dataset to assemble `PREOP`, `PREOP_TREE`, `POSTOP` and `POSTOP_TREE` feature sets as appropriate
- these are essentially helper lists as `DataFrames` put through `Pipelines` emerge as `numpy` arrays which do not have column names
- can convert resulting `numpy` arrays back into `DataFrames` for further analysis

#### `PREOP_dataset` Column Names

- `DUMMIES` Version = `PREOP_numerical_col_names` + `PREOP_categorical_col_names`
- `TREE` Version    = `PREOP_numerical_col_names` + `PREOP_categorical_TREE_col_names`

- `PREOP_numerical_col_names`

In [79]:
with open('PREOP_numerical_col_names_11_14.pkl', 'rb') as filehandle:
    PREOP_numerical_col_names = pickle.load(filehandle)

In [80]:
PREOP_numerical_col_names[0:5]

['age', 'heightcm', 'weightkg', 'bmi', 'hct']

In [81]:
len(PREOP_numerical_col_names)

11

- `PREOP_categorical_col_names`

In [82]:
with open('PREOP_categorical_col_names_11_14.pkl', 'rb') as filehandle:
    PREOP_categorical_col_names = pickle.load(filehandle)

In [83]:
PREOP_categorical_col_names[0:5]

['surgdt_month_Jan',
 'surgdt_month_Feb',
 'surgdt_month_Mar',
 'surgdt_month_Apr',
 'surgdt_month_May']

In [84]:
len(PREOP_categorical_col_names)

99

- `PREOP_categorical_TREE_col_names`

In [85]:
with open('PREOP_categorical_TREE_col_names_11_14.pkl', 'rb') as filehandle:
    PREOP_categorical_TREE_col_names = pickle.load(filehandle)

In [86]:
PREOP_categorical_TREE_col_names[0:5]

['surgdt_month',
 'surgdt_DayOfWeek',
 'surgdt_PartOfMonth',
 'gender',
 'racecaucasian']

In [87]:
len(PREOP_categorical_TREE_col_names)

61

#### `POSTOP_dataset` Column Names

- `DUMMIES` Version = `POSTOP_numerical_col_names` + `POSTOP_categorical_col_names`
- `TREE` Version = `POSTOP_numerical_col_names` + `POSTOP_categorical_TREE_col_names`

In [88]:
with open('POSTOP_numerical_col_names_11_14.pkl', 'rb') as filehandle:
    POSTOP_numerical_col_names = pickle.load(filehandle)

In [89]:
with open('POSTOP_categorical_col_names_11_14.pkl', 'rb') as filehandle:
    POSTOP_categorical_col_names = pickle.load(filehandle)

In [90]:
with open('POSTOP_categorical_TREE_col_names_11_14.pkl', 'rb') as filehandle:
    POSTOP_categorical_TREE_col_names = pickle.load(filehandle)

## Functions Used in Pipelines

#### `SimpleImputer(missing_values=np.nan, strategy='median'` 
- to replace numerical feature `NaN`s with `X_train` numerical feature `NaN`s

#### `PolynomialFeatures()`
- per Albon 13.2 pages 225-227, can use `PolynomialFeatures()` to create interaction terms
- Parameter `interaction_only=True` tells `PolynomialFeatures()` to ONLY return interaction terms (and not polynomial features
- By default, `PolynomialFeatures()` will add a feature containing ones (a vector of `1`s) called a `bias` -- we can prevent that through the parameter `include_bias=False`
- The `degree` parameter determines the maximum number of features to create interaction terms from (in case we wanted to create an interaction term of `n_features`)
- open question whether we should apply `PolynomialFeatures` to the entire feature matrix or only to the numerical and/or categorical features and how we should do it (with everything higher degree + interaction terms or just interaction terms).
- Given the number of featues we have, we will have many more features after transforming with `PolynomialFeatures`
- The formula for calculating the number of the polynomial features is `N(n,d)=C(n+d,d)` where `n` is the number of the features, `d` is the degree of the polynomial, `C` is binomial coefficient(combination). In our case the number is `C(3+2,2)=5!/(5-2)!2!=10` but when the number of features or the degree is high the polynomial features becomes too many.

#### `sklearn.preprocessing` scalers such as `StandardScaler()`

#### Custom `helper` functions, such as `convert_df_to_numpy`
- need to create a function to convert `DataFrame` to `numpy.ndarray` then can use `FeatureUnion` to combine with `numpy.ndarray` that results from `numerical_pipeline` to form feature matrix

In [91]:
def convert_df_to_numpy(df):
    
    df = df.values
    
    return df

#### `FeatureUnion`
- to reassemble feature matrix from the product of multiple `pipelines`

# `PREOP` Feature Matrix Pipelines

### `PREOP_numerical_features` Pipeline

In [92]:
PREOP_numerical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(PREOP_numerical_col_names)), 
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
    ('scaler', StandardScaler()) 
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [93]:
PREOP_numerical = PREOP_numerical_features_pipeline.fit_transform(X_train_all)

In [94]:
PREOP_numerical.shape, len(PREOP_numerical_col_names)

((4236, 11), 11)

### `PREOP_categorical_features_pipeline`

In [95]:
PREOP_categorical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(PREOP_categorical_col_names)),
    ('convert_numpy', FunctionTransformer(convert_df_to_numpy, validate=False))
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [96]:
PREOP_categorical = PREOP_categorical_features_pipeline.fit_transform(X_train_all)

In [97]:
PREOP_categorical.shape, len(PREOP_categorical_col_names)

((4236, 99), 99)

## Assembling `PREOP_feature_matrix` via `FeatureUnion`

In [98]:
PREOP_feature_matrix = FeatureUnion([('PREOP_num_features', PREOP_numerical_features_pipeline),
                                     ('PREOP_cat_features', PREOP_categorical_features_pipeline)
                                    ])

In [99]:
type(PREOP_feature_matrix)

sklearn.pipeline.FeatureUnion

### Transforming `X_train_all` to get `X_train_PREOP`

In [100]:
X_train_PREOP = PREOP_feature_matrix.fit_transform(X_train_all)

- validating

In [101]:
X_train_PREOP.shape, y_train.shape

((4236, 110), (4236,))

- putting into a `DataFrame` for easy analysis

In [102]:
PREOP_feature_col_names = PREOP_numerical_col_names + PREOP_categorical_col_names

In [103]:
len(PREOP_feature_col_names), len(PREOP_numerical_col_names), len(PREOP_categorical_col_names)

(110, 11, 99)

In [104]:
X_train_PREOP = pd.DataFrame(X_train_PREOP,
                             columns=PREOP_feature_col_names)

In [105]:
X_train_PREOP.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%
0,0.91695,-0.21828,1.55270,0.87201,0.54584,0.73343,0.24251,-0.27709,1.35152,0.19560,-1.79947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.62237,0.68395,0.41176,-0.01820,-1.30610,-0.16478,-0.68918,0.23397,-0.62449,-2.02718,-0.03218,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.62237,-1.62577,-0.45636,0.24439,-0.36197,0.35918,-0.13017,2.87445,0.66367,0.34379,-0.70544,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.62237,0.20577,-0.81352,-0.49794,0.25534,-0.08057,0.24251,-0.27709,-0.33685,-0.91579,1.14601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,-0.62237,0.23283,-0.67959,-0.43646,-0.36197,-0.38933,0.05617,0.14880,-0.33685,0.56607,-0.70544,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Transforming `X_dev_all` to get `X_dev_PREOP`

In [106]:
X_dev_PREOP = PREOP_feature_matrix.transform(X_dev_all)

- validating

In [107]:
X_dev_PREOP.shape, y_dev.shape

((529, 110), (529,))

- putting into a `DataFrame` for easy analysis

In [108]:
X_dev_PREOP = pd.DataFrame(X_dev_PREOP,
                           columns=PREOP_feature_col_names)

In [109]:
X_dev_PREOP.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%
0,1.43006,0.68395,-0.03470,-0.23135,0.50953,-0.51096,-0.50284,-0.44744,-0.29620,0.93653,0.47276,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.93023,-0.47993,-0.61014,-0.21612,-1.45135,0.17205,0.05617,-1.12886,1.15767,-1.87899,-0.45297,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.50647,-0.01979,-1.11612,-0.60286,-0.30751,-0.37062,1.54687,0.23397,-0.74331,1.30699,-0.78959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.30123,0.68395,0.11412,-0.16030,0.90897,0.17205,-0.13017,0.23397,0.25721,-2.02718,1.73511,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.13547,0.41328,-0.53077,-0.40512,-0.90666,-0.20220,-0.31650,-0.78815,-0.61824,-0.32305,1.31433,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Transforming `X_test_all` to get `X_test_PREOP`

In [110]:
X_test_PREOP = PREOP_feature_matrix.transform(X_test_all)

- validating

In [111]:
X_test_PREOP.shape, y_test.shape

((530, 110), (530,))

- putting into a `DataFrame` for easy analysis

In [112]:
X_test_PREOP = pd.DataFrame(X_test_PREOP,
                            columns=PREOP_feature_col_names)

In [113]:
X_test_PREOP.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%
0,1.12220,1.31552,0.16372,-0.29662,0.72740,-0.10864,-0.13017,-0.02156,-0.46191,0.19560,-0.03218,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.60909,-0.94007,-0.86313,-0.23117,0.36428,-0.38933,0.80152,0.23397,-0.43064,0.56607,-0.12476,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.60909,0.66591,-0.11903,-0.26706,0.00115,-0.20220,-0.50284,-1.12886,-0.74331,-0.32305,-0.62128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.62237,-1.39119,-1.32447,-0.38656,0.18271,-0.20220,1.73320,-0.70297,-0.74331,0.56607,-0.12476,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.50647,-0.66940,-0.28273,0.02411,-0.18041,-0.29577,-0.13017,0.06362,-0.74331,0.56607,-0.12476,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### NOTE:
- did not have to put feature matricies into `DataFrames` - could have used directly in pipelines, but wanted to put into `DataFrames` in case wanted to retrieve feature names

# `POSTOP` Feature Matrix Pipelines

### `POSTOP_numerical_features` Pipeline

In [114]:
POSTOP_numerical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(POSTOP_numerical_col_names)), 
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
    ('scaler', StandardScaler()) 
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [115]:
POSTOP_numerical = POSTOP_numerical_features_pipeline.fit_transform(X_train_all)

In [116]:
POSTOP_numerical.shape, len(POSTOP_numerical_col_names)

((4236, 15), 15)

### `POSTOP_categorical_features_pipeline`

In [117]:
POSTOP_categorical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(POSTOP_categorical_col_names)),
    ('convert_numpy', FunctionTransformer(convert_df_to_numpy, validate=False))
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [118]:
POSTOP_categorical = POSTOP_categorical_features_pipeline.fit_transform(X_train_all)

In [119]:
POSTOP_categorical.shape, len(POSTOP_categorical_col_names)

((4236, 103), 103)

## Assembling `POSTOP_feature_matrix` via `FeatureUnion`

In [120]:
POSTOP_feature_matrix = FeatureUnion([('POSTOP_num_features', POSTOP_numerical_features_pipeline),
                                      ('POSTOP_cat_features', POSTOP_categorical_features_pipeline)
                                     ])

In [121]:
type(POSTOP_feature_matrix)

sklearn.pipeline.FeatureUnion

### Transforming `X_train_all` to get `X_train_POSTOP`

In [122]:
X_train_POSTOP = POSTOP_feature_matrix.fit_transform(X_train_all)

- validating

In [123]:
X_train_POSTOP.shape, y_train.shape

((4236, 118), (4236,))

- putting into a `DataFrame` for easy analysis

In [124]:
POSTOP_feature_col_names = POSTOP_numerical_col_names + POSTOP_categorical_col_names

In [125]:
len(POSTOP_feature_col_names), len(POSTOP_numerical_col_names), len(POSTOP_categorical_col_names)

(118, 15, 103)

In [126]:
X_train_POSTOP = pd.DataFrame(X_train_POSTOP,
                              columns=POSTOP_feature_col_names)

In [127]:
X_train_POSTOP.head()

,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,mtcause_Cardiac,mtcause_Infection,mtcause_Neuro,mtcause_Pulmonary,mtcause_Renal,mtcause_Vascular,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn
0,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,0.47425,0.45658,-0.31952,0.14889,0.02817,0.03293,-0.27281,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,-1.04086,0.65426,-0.07228,-0.07432,0.02817,0.03293,0.17717,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,-0.81805,-0.72954,0.28091,0.00009,0.02817,0.03293,0.64964,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,0.51881,1.69211,0.17495,-0.37193,0.02817,0.03293,0.49215,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-

### Transforming `X_dev_all` to get `X_dev_POSTOP`

In [128]:
X_dev_POSTOP = POSTOP_feature_matrix.transform(X_dev_all)

- validating

In [129]:
X_dev_POSTOP.shape, y_dev.shape

((529, 118), (529,))

- putting into a `DataFrame` for easy analysis

In [130]:
X_dev_POSTOP = pd.DataFrame(X_dev_POSTOP,
                            columns=POSTOP_feature_col_names)

In [131]:
X_dev_POSTOP.head()

,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,mtcause_Cardiac,mtcause_Infection,mtcause_Neuro,mtcause_Pulmonary,mtcause_Renal,mtcause_Vascular,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn
0,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,1.40194,0.47704,-1.65040,0.51881,1.19790,0.59879,-0.37193,0.02817,0.03293,-1.08276,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.84738,-0.59524,-0.33417,-1.53804,0.07449,0.02817,0.03293,-1.48774,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,1.31138,-0.04436,-0.90718,-0.63069,-0.88463,-0.52073,0.02817,0.03293,-0.74528,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,1.40194,0.47704,-1.65040,1.05356,0.16005,-0.39016,0.00009,0.02817,0.03293,-0.34030,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-

### Transforming `X_test_all` to get `X_test_POSTOP`

In [132]:
X_test_POSTOP = POSTOP_feature_matrix.transform(X_test_all)

- validating

In [133]:
X_test_POSTOP.shape, y_test.shape

((530, 118), (530,))

- putting into a `DataFrame` for easy analysis

In [134]:
X_test_POSTOP = pd.DataFrame(X_test_POSTOP,
                             columns=POSTOP_feature_col_names)

In [135]:
X_test_POSTOP.head()

,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,mtcause_Cardiac,mtcause_Infection,mtcause_Neuro,mtcause_Pulmonary,mtcause_Renal,mtcause_Vascular,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn
0,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,0.29600,0.16005,0.28091,-0.22312,0.02817,0.03293,0.67214,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,-0.37243,0.70368,-1.52038,-0.59514,0.02817,0.03293,-1.55523,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,-0.14962,0.20947,-0.33718,-0.44633,0.02817,0.03293,-0.16031,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,0.74163,-0.23532,-0.39016,-0.66954,0.02817,0.03293,-0.49779,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33

## Assembling `PREOP + POSTOP` Combined Feature Matrix

#### -  Using `FeatureUnion | Pipeline` to Create Combined `PREOP + POSTOP` Feature Matrix

In [136]:
PREPOST_feature_matrix = FeatureUnion([('PREOP_num_features', PREOP_numerical_features_pipeline),
                                       ('PREOP_cat_features', PREOP_categorical_features_pipeline),
                                       ('POSTOP_num_features', POSTOP_numerical_features_pipeline),
                                       ('POSTOP_cat_features', POSTOP_categorical_features_pipeline)
                                      ])

- can use the code pattern above to create `X_train_PREPOST`, `X_dev_PREPOST`, `X_test_PREPOST`

In [137]:
PREPOST_feature_col_names = PREOP_feature_col_names + POSTOP_feature_col_names

In [138]:
len(PREPOST_feature_col_names), len(PREOP_feature_col_names), len(POSTOP_feature_col_names)

(228, 110, 118)

### Transforming `X_train_all` to get `X_train_PREPOST`

In [139]:
X_train_PREPOST = PREPOST_feature_matrix.fit_transform(X_train_all)

- validating

In [140]:
X_train_PREPOST.shape, y_train.shape

((4236, 228), (4236,))

- putting into a `DataFrame` for easy analysis

In [141]:
X_train_PREPOST = pd.DataFrame(X_train_PREPOST,
                               columns=PREPOST_feature_col_names)

In [142]:
X_train_PREPOST.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,mtcause_Cardiac,mtcause_Infection,mtcause_Neuro,mtcause_Pulmonary,mtcause_Renal,mtcause_Vascular,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn
0,0.91695,-0.21828,1.55270,0.87201,0.54584,0.73343,0.24251,-0.27709,1.35152,0.19560,-1.79947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,0.47425,0.45658,-0.31952,0.14889,0.02817,0.03293,-0.27281,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0

### Transforming `X_dev_all` to get `X_dev_PREPOST`

In [143]:
X_dev_PREPOST = PREPOST_feature_matrix.transform(X_dev_all)

- validating

In [144]:
X_dev_PREPOST.shape, y_dev.shape

((529, 228), (529,))

- putting into a `DataFrame` for easy analysis

In [145]:
X_dev_PREPOST = pd.DataFrame(X_dev_PREPOST,
                             columns=PREPOST_feature_col_names)

In [146]:
X_dev_PREPOST.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,mtcause_Cardiac,mtcause_Infection,mtcause_Neuro,mtcause_Pulmonary,mtcause_Renal,mtcause_Vascular,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn
0,1.43006,0.68395,-0.03470,-0.23135,0.50953,-0.51096,-0.50284,-0.44744,-0.29620,0.93653,0.47276,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,1.40194,0.47704,-1.65040,0.51881,1.19790,0.59879,-0.37193,0.02817,0.03293,-1.08276,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.

### Transforming `X_test_all` to get `X_test_PREPOST`

In [147]:
X_test_PREPOST = PREPOST_feature_matrix.transform(X_test_all)

- validating

In [148]:
X_test_PREPOST.shape, y_test.shape

((530, 228), (530,))

- putting into a `DataFrame` for easy analysis

In [149]:
X_test_PREPOST = pd.DataFrame(X_test_PREPOST,
                              columns=PREPOST_feature_col_names)

In [150]:
X_test_PREPOST.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,mtcause_Cardiac,mtcause_Infection,mtcause_Neuro,mtcause_Pulmonary,mtcause_Renal,mtcause_Vascular,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn
0,1.12220,1.31552,0.16372,-0.29662,0.72740,-0.10864,-0.13017,-0.02156,-0.46191,0.19560,-0.03218,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,0.29600,0.16005,0.28091,-0.22312,0.02817,0.03293,0.67214,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1

## Modeling Scenarios Using `PREOP` and `POSTOP` Datasets

### - Scenario A: `Kitchen Sink` - Modeling the Combined `PREOP` and `POSTOP` Dataset

- code belows shows feature matrix assembly using `Pipeline` - could have used the already named `feature matricies` created above: `X_train_PREPOST`, `X_dev_PREPOST` and `X_test_PREPOST` in conjunction with their respective `y` vectors

#### `LogisticRegression()` with `GridSearchCV`

- instantiating `Pipeline`

In [151]:
PREPOST_modelA_pipe = Pipeline(steps=[
    ('get_PREPOST_feature_matrix', PREPOST_feature_matrix),
    ('Classifier', LogisticRegression())
    ])

- defining a parameter grid for `GridSearchCV`

In [152]:
parameter_grid = {'Classifier__penalty': ['l1'],
                  'Classifier__C': [0.001, 0.01, 1.0],
                  'Classifier__class_weight': ['balanced'],
                  'Classifier__solver': ['liblinear'],
                  'Classifier__random_state': [0]
                 }

- defining `scoring_metrics`

In [153]:
scoring_metrics = ['accuracy',
                   'f1',
                   'f1_macro',
                   'f1_weighted',
                   'precision',
                   'precision_macro',
                   'precision_weighted',
                   'recall',
                   'recall_macro',
                   'recall_weighted',
                   'roc_auc']

- create `GridSearchCV` object

In [154]:
gs = GridSearchCV(PREPOST_modelA_pipe, #pipline
                  param_grid=parameter_grid, #param_grid
                  scoring=scoring_metrics, #loss functions
                  cv=5, #number of folds, default=`StratifiedKFold`
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  return_train_score=True,
                  n_jobs=-1,
                  verbose=5)

- fitting `GridSearchCV`

In [155]:
PREPOST_modelA = gs.fit(X_train_all, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:   12.1s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   15.7s finished


- examining results

In [156]:
gs_results = pd.DataFrame(PREPOST_modelA.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                  ascending=False)

In [157]:
display_cols = ['param_Classifier__C',
                'param_Classifier__penalty',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']

In [158]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)

In [159]:
gs_results_summary.rename(columns={'param_Classifier__C': 'C',
                                   'param_Classifier__penalty': 'penalty'},
                          inplace=True)

In [160]:
gs_results_summary

,C,penalty,mean_test_roc_auc,mean_train_roc_auc,mean_test_f1,mean_test_precision,mean_test_recall
0,0.01,l1,0.65251,0.70855,0.08581,0.04636,0.57597
1,1,l1,0.60210,0.92063,0.08730,0.05056,0.32129
2,0.001,l1,0.50000,0.50000,0.00000,0.00000,0.00000


#### Key Takeaways
- `LogisticRegression()` with `C=0.01` beats `STS`

## Scenario Ab - Just Preop Data Logistic

### Step 3b: Model and Evaluate Stacked Model Using `RandomForestClassifier`

In [221]:
PRE_feature_matrix = FeatureUnion([('PREOP_num_features', PREOP_numerical_features_pipeline),
                                       ('PREOP_cat_features', PREOP_categorical_features_pipeline)
                                      ])

In [222]:
PRE_modelA_pipe = Pipeline(steps=[
    ('get_PRE_feature_matrix', PRE_feature_matrix),
    ('Classifier', LogisticRegression())
    ])

In [223]:
parameter_grid = {'Classifier__penalty': ['l1'],
                  'Classifier__C': [0.0275, 0.01, 1.0],
                  'Classifier__class_weight': ['balanced'],
                  'Classifier__solver': ['liblinear'],
                  'Classifier__random_state': [0]
                 }

In [224]:
scoring_metrics = ['accuracy',
                   'f1',
                   'f1_macro',
                   'f1_weighted',
                   'precision',
                   'precision_macro',
                   'precision_weighted',
                   'recall',
                   'recall_macro',
                   'recall_weighted',
                   'roc_auc']

In [225]:
gs = GridSearchCV(PRE_modelA_pipe, #pipline
                  param_grid=parameter_grid, #param_grid
                  scoring=scoring_metrics, #loss functions
                  cv=5, #number of folds, default=`StratifiedKFold`
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  return_train_score=True,
                  n_jobs=-1,
                  verbose=5)

In [226]:
PRE_modelA = gs.fit(X_train_all, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:    6.7s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    7.6s finished


In [227]:
gs_results = pd.DataFrame(PRE_modelA.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                  ascending=False)
display_cols = ['param_Classifier__C',
                'param_Classifier__penalty',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)
gs_results_summary.rename(columns={'param_Classifier__C': 'C',
                                   'param_Classifier__penalty': 'penalty'},
                          inplace=True)

In [228]:
gs_results_summary

,C,penalty,mean_test_roc_auc,mean_train_roc_auc,mean_test_f1,mean_test_precision,mean_test_recall
0,0.0275,l1,0.64898,0.75749,0.08682,0.04744,0.51228
1,1,l1,0.61026,0.85568,0.09022,0.05017,0.44861
2,0.01,l1,0.60085,0.66148,0.06987,0.03735,0.54337


### - Scenario B: `Model Stacking` - Incorporate Predictions from `PREOP` Training with `POSTOP` Data
- this time, we will make use of pre-saved `X_train_PREOP` and `X_train_POSTOP` feature matricies created above via `Pipeline` 

In [161]:
X_train_PREOP.shape, X_train_POSTOP.shape, y_train.shape

((4236, 110), (4236, 118), (4236,))

#### Step 1.  Generate vector of probabilities of stroke for using `PREOP` features and cross validated training set
- `LogisticRegression` hyperparameters were tuned in a separate notebook

In [162]:
lr_predict_probs = cross_val_predict(LogisticRegression(penalty='l1',
                                                        C=0.0275,
                                                        class_weight='balanced',
                                                        random_state=0,
                                                        solver='liblinear'),
                                     X_train_PREOP,
                                     y_train,
                                     cv=5,
                                     method='predict_proba',
                                     n_jobs=-1,
                                     verbose=5)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


- `cross_val_predict` returns an `numpy.ndarray` with `shape` `(n, 2)` as it returns a probability per class `(0, 1)`
- since we are interested in the `1` class, need to carve our the second column of the `numpy.ndarray`
- add `[:,1]`

- examining `lr_predict_probs`

In [163]:
type(lr_predict_probs)

numpy.ndarray

In [164]:
lr_predict_probs.shape

(4236, 2)

In [165]:
lr_predict_probs[0:5]

array([[0.71599274, 0.28400726],
       [0.53830776, 0.46169224],
       [0.36722051, 0.63277949],
       [0.31917245, 0.68082755],
       [0.65599598, 0.34400402]])

In [166]:
lr_predict_probs[0:5, 1]

array([0.28400726, 0.46169224, 0.63277949, 0.68082755, 0.34400402])

### Step 1a.  Generate additional vector of probabilities of stroke for using `PREOP` features and CV training set
- Not limited to stacking one `PREOP` model
- Running `RandomForestClassifier` `PREOP` model 
- hyperparameters were tuned in a separate notebook - using AP's highest scoring model
- going to assemble `X_train_PREOP_TREE` using `Pipelines`
- can utilize existing `PREOP_numerical_features_pipeline`
- need to define `PREOP_categorial_TREE_features_pipeline`
- use`FeatureUnion` to combine to create `PREOP` feature matrix usable for `RandomForestClassifier`

- creating `Pipeline` to assemble `PREOP_categorical_TREE_features_pipeline`

In [251]:
PREOP_categorical_TREE_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(PREOP_categorical_TREE_col_names)),
    ('convert_numpy', FunctionTransformer(convert_df_to_numpy, validate=False))
    ])

In [252]:
PREOP_feature_matrix_TREE = FeatureUnion([('PREOP_num_features', PREOP_numerical_features_pipeline),
                                          ('PREOP_cat_tree_features', PREOP_categorical_TREE_features_pipeline)
                                         ])

- combining with `PREOP_numerical_features_pipeline` defined above to assemble `PREOP_feature_matrix_TREE`
- if plan is to assemble `feature_matrix` but not use in a modeling pipeline as is done below, then can assign to a variable name like `X_train_PREOP_TREE` -- did this above for `X_train_PREOP` 

#### Modeling `PREOP_dataset_TREE` using `RandomForestClassifier()`

- creating `PREOP_RF_CLF_pipe`

In [253]:
PREOP_RF_clf_pipe = Pipeline(steps=[
    ('get_PREOP_TREE_feature_matrix', PREOP_feature_matrix_TREE),
    ('Classifier', RandomForestClassifier(n_estimators=315,
                                          max_features=0.1,
                                          max_depth=None,
                                          min_samples_split=50,
                                          class_weight='balanced',
                                          random_state=0))])

- in this scenario - assuming already have tuned the hyperparamenters
- alternatively, could have run a `GridSearchCV` first to tune the hyperparameters and then use the `best_estimator_` to get estimated probabilities of stroke with the code below

- running `RandomForestClassifer`

In [254]:
rf_predict_probs = cross_val_predict(PREOP_RF_clf_pipe,
                                     X_train_all,
                                     y_train,
                                     cv=5,
                                     method='predict_proba',
                                     n_jobs=-1,
                                     verbose=5)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   12.4s remaining:   18.6s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   15.9s finished


- examining `rf_predict_probs`

In [255]:
rf_predict_probs.shape

(4236, 2)

In [256]:
rf_predict_probs[0:5, 1]

array([0.12992653, 0.22474483, 0.3199263 , 0.25008449, 0.15170196])

### Step 2: Incorporate vector of predicted probabilities into `X_train_POSTOP`
- adding `lr_predict_probs`
- adding `rf_predict_probs`
- adding `STS` model predicted probability using `predstro_train`

In [257]:
X_train_POSTOP_STACKED = X_train_POSTOP.copy()

In [258]:
X_train_POSTOP_STACKED.shape, X_train_POSTOP.shape

((4236, 118), (4236, 118))

- adding vectors of `predict_probas` from `PREOP` models and `STS` models to `POSTOP` feature matrix

In [259]:
X_train_POSTOP_STACKED['LR_PREOP_PROBA'] = lr_predict_probs[:, 1]
X_train_POSTOP_STACKED['RF_PREOP_PROBA'] = rf_predict_probs[:, 1]
#X_train_POSTOP_STACKED['STS_predstro'] = predstro_train

In [260]:
X_train_POSTOP_STACKED.head()

,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,mtcause_Cardiac,mtcause_Infection,mtcause_Neuro,mtcause_Pulmonary,mtcause_Renal,mtcause_Vascular,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,LR_PREOP_PROBA,RF_PREOP_PROBA
0,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,0.47425,0.45658,-0.31952,0.14889,0.02817,0.03293,-0.27281,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.28401,0.12993
1,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,-1.04086,0.65426,-0.07228,-0.07432,0.02817,0.03293,0.17717,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.46169,0.22474
2,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,-0.81805,-0.72954,0.28091,0.00009,0.02817,0.03293,0.64964,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.63278,0.31993
3,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,0.51881,1.69211,0.17495,-0.37193,0.02817,0.03293,0.49215,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [261]:
X_train_POSTOP_STACKED.shape

(4236, 120)

### Step 3a: Model and Evaluate `X_train_POSTOP_STACKED` Using `LogisticRegression()`

- instantiating a `LogisticRegression()` object

In [262]:
log_reg_clf = LogisticRegression()

- will use scoring metrics defined above in previous `GridSearchCV`
- defining hyperparameter grid

In [263]:
hyperparameters = {'penalty': ['l1'],
                   'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0],
                   'class_weight': ['balanced'],
                   'solver': ['liblinear'],
                   'random_state': [0]
                  }

- create `GridSearchCV` object

In [264]:
gs_stacked = GridSearchCV(log_reg_clf, #classifier
                          param_grid=hyperparameters, #param_grid
                          scoring=scoring_metrics, #loss functions
                          cv=5, #number of folds, default=`StratifiedKFold`
                          refit='roc_auc', # best_estimator_ will be based on this scoring metric
                          return_train_score=True,
                          n_jobs=-1,
                          verbose=5)

- fitting `GridSearchCV`

In [265]:
PREPOST_modelB = gs_stacked.fit(X_train_POSTOP_STACKED, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   46.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   46.6s finished


- examining results

In [266]:
gs_results = pd.DataFrame(PREPOST_modelB.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                      ascending=False)

In [267]:
display_cols = ['param_C',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']

In [268]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)

In [269]:
gs_results_summary

,param_C,mean_test_roc_auc,mean_train_roc_auc,mean_test_f1,mean_test_precision,mean_test_recall
0,0.1,0.65859,0.77173,0.09440,0.05227,0.48774
1,1,0.61699,0.81232,0.09855,0.05494,0.47911
2,10,0.59922,0.82007,0.09429,0.05260,0.45572
3,0.01,0.58627,0.62506,0.06995,0.03806,0.44098
4,0.0001,0.50000,0.50000,0.00000,0.00000,0.00000
5,0.001,0.50000,0.50000,0.00000,0.00000,0.00000


### Key Takeaways
- similar to `Modeling Scenario A` achieved better results than `STS` with the `POSTOP` features
- results were slightly below that of `Modeling Scenario A` on a `mean_test_roc_auc` basis
- extend this stacking concept to add additional vectors of probability of stroke from other models `RandomForest` and `STS`
- adding the `RandomForest` predicted probabilities improved results slightly, but the addition of `STS` on top of that did not improve the `mean_test_roc_auc`
- instead of modeling the stacked `POSTOP` data with `LogisticRegression` can try another model such as `RandomForest` or `SVM`
- in addition can also try using `PolynomialFeatures` transformation using `Scenario B` to see if any benefit from modeling the interaction of the `PREOP` predictions with the additional `POSTOP` features
- should examine the non-zero coeficients in `POSTOP` to get insight into what `POSTOP` features are useful in enhancing stroke prediction

## Step 3 ab - Just Preop Data RF

In [229]:
PREOP_RF_clf_pipe = Pipeline(steps=[
    ('get_PREOP_TREE_feature_matrix', PREOP_feature_matrix_TREE),
    ('Classifier', RandomForestClassifier())])

In [230]:
parameter_grid = {'Classifier__n_estimators': [315],
                  'Classifier__max_features': [0.1],
                  'Classifier__max_depth': [None],
                  'Classifier__min_samples_split': [50],
                  'Classifier__class_weight': ['balanced'],
                  'Classifier__random_state': [0]
                 }
scoring_metrics = ['accuracy',
                   'f1',
                   'f1_macro',
                   'f1_weighted',
                   'precision',
                   'precision_macro',
                   'precision_weighted',
                   'recall',
                   'recall_macro',
                   'recall_weighted',
                   'roc_auc']

In [231]:
gs = GridSearchCV(PREOP_RF_clf_pipe, #pipline
                  param_grid=parameter_grid, #param_grid
                  scoring=scoring_metrics, #loss functions
                  cv=5, #number of folds, default=`StratifiedKFold`
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  return_train_score=True,
                  n_jobs=-1,
                  verbose=5)

In [232]:
PRE_modelRF = gs.fit(X_train_all, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    7.8s remaining:   11.7s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   11.8s finished


In [233]:
gs_results = pd.DataFrame(PRE_modelRF.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                  ascending=False)
display_cols = ['param_Classifier__n_estimators',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)
gs_results_summary.rename(columns={'param_Classifier__n_estimators': 'n_estimators'},
                          inplace=True)

In [234]:
gs_results_summary

,n_estimators,mean_test_roc_auc,mean_train_roc_auc,mean_test_f1,mean_test_precision,mean_test_recall
0,315,0.62883,0.99996,0.0,0.0,0.0


### Step 3b: Model and Evaluate Stacked Model Using `RandomForestClassifier`

#### Creating `POSTOP_feature_matrix_TREE` using `sklearn` `Pipelines` 

- can use similar coding pattern to assemble `POSTOP_feature_matrix_TREE` if needed
    - `POSTOP_numerical_features_pipeline` has been previously created above
    - would need to create `POSTOP_categorical_TREE_features_pipeline` -- THIS HAS NOT BEEN DONE YET
    - then use `FeatureUnion()` to assemble `POSTOP_feature_matrix_TREE`
    - similar to what was done above in using `Pipeline`s to create `X_train_PREOP`, `X_train_POSTOP` and corresponding `X_dev` and `X_test` 

In [270]:
POSTOP_categorical_TREE_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(POSTOP_categorical_TREE_col_names)),
    ('convert_numpy', FunctionTransformer(convert_df_to_numpy, validate=False))
    ])

- using `FeatureUnion` combining using with already created `POSTOP_numerical_features_pipeline` to assemble `POSTOP_feature_matrix_TREE`

In [271]:
POSTOP_feature_matrix_TREE = FeatureUnion([('POSTOP_num_features', POSTOP_numerical_features_pipeline),
                                           ('POSTOP_cat_tree_features', POSTOP_categorical_TREE_features_pipeline)
                                          ])

#### NOTE - as we did above:

- `fit_transform` only on `X_train_all` for `X_train_POSTOP_TREE`
- `transform` on `X_dev_all` and `X_test_all` for `X_dev_POSTOP_TREE` and `X_test_POSTOP_TREE`
- assign compreleted `feature_matrix` to variable name if not using directly in a `Pipeline` with classifier step

#### Transforming `X_train_all` to get `X_train_POSTOP_TREE`

In [272]:
X_train_POSTOP_TREE = POSTOP_feature_matrix_TREE.fit_transform(X_train_all)

- validating

In [273]:
X_train_all.shape, X_train_POSTOP_TREE.shape, y_train.shape

((4236, 255), (4236, 64), (4236,))

- putting into a `DataFrame` for easy analysis

In [274]:
POSTOP_feature_TREE_col_names = POSTOP_numerical_col_names + POSTOP_categorical_TREE_col_names

In [275]:
len(POSTOP_feature_TREE_col_names), len(POSTOP_numerical_col_names), len(POSTOP_categorical_TREE_col_names)

(64, 15, 49)

In [276]:
X_train_POSTOP_TREE = pd.DataFrame(X_train_POSTOP_TREE,
                                   columns=POSTOP_feature_TREE_col_names)

In [277]:
X_train_POSTOP_TREE.head()

,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,mtcause,unplproc,urgntrsn
0,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,0.47425,0.45658,-0.31952,0.14889,0.02817,0.03293,-0.27281,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,-1.04086,0.65426,-0.07228,-0.07432,0.02817,0.03293,0.17717,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,-0.81805,-0.72954,0.28091,0.00009,0.02817,0.03293,0.64964,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,0.51881,1.69211,0.17495,-0.37193,0.02817,0.03293,0.49215,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,-0.37243,0.65426,-0.54909,-0.37193,0.02817,0.03293,-0.54279,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


#### Creating `X_train_POSTOP_TREE_STACKED`
- adding vectors of `predict_probas` from `PREOP` models and `STS` models to `POSTOP_TREE` feature matrix

In [278]:
X_train_POSTOP_TREE_STACKED = X_train_POSTOP_TREE.copy()

In [279]:
X_train_POSTOP_TREE_STACKED.shape, X_train_POSTOP_TREE.shape

((4236, 64), (4236, 64))

In [280]:
X_train_POSTOP_TREE_STACKED['LR_PREOP_PROBA'] = lr_predict_probs[:, 1]
X_train_POSTOP_TREE_STACKED['RF_PREOP_PROBA'] = rf_predict_probs[:, 1]
#X_train_POSTOP_TREE_STACKED['STS_predstro'] = predstro_train

In [281]:
X_train_POSTOP_TREE_STACKED.head()

,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,mtopd,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,mtcause,unplproc,urgntrsn,LR_PREOP_PROBA,RF_PREOP_PROBA
0,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,0.47425,0.45658,-0.31952,0.14889,0.02817,0.03293,-0.27281,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.28401,0.12993
1,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,-1.04086,0.65426,-0.07228,-0.07432,0.02817,0.03293,0.17717,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.46169,0.22474
2,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,-0.81805,-0.72954,0.28091,0.00009,0.02817,0.03293,0.64964,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.63278,0.31993
3,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,0.51881,1.69211,0.17495,-0.37193,0.02817,0.03293,0.49215,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.68083,0.25008
4,-0.00525,-0.07508,-0.09078,0.01012,-0.12913,-0.33399,-0.35729,-0.04436,-0.37243,0.65426,-0.54909,-0.37193,0.02817,0.03293,-0.54279,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.34400,0.15170


In [282]:
X_train_POSTOP_TREE_STACKED.shape

(4236, 66)

- instantiating a `RandomForestClassifier()` object
- code pattern from line 169 - `GridSearchCV`

In [283]:
rf_clf = RandomForestClassifier(class_weight='balanced',
                                random_state=0)

In [284]:
hyper_params = {'n_estimators': [100, 200, 250, 300],          # default is `10`
                'max_features': ['auto', 0.10, 0.25, 0.50],    # default is square root of `n_features`    
                'max_depth': [2, 5, 10, None],                 # default is `None`
                'min_samples_split': [2, 5, 10]}               # default is `2`

In [285]:
gs = GridSearchCV(rf_clf, # estimator
                  hyper_params, # param_grid
                  scoring=scoring_metrics, # loss functions
                  cv=5, 
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  verbose=2,
                  return_train_score=True,
                  n_jobs=-1)

In [286]:
rf_gs_stacked_model = gs.fit(X_train_POSTOP_TREE_STACKED, 
                             y_train)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   52.6s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 13.2min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 27.7min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed: 48.2min finished
/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


- `GridSearchCV` takes approximately 90 minutes to run

#### Saving `rf_gs_stacked_model`

In [287]:
joblib.dump(rf_gs_stacked_model, 'rf_gs_stacked_model_cabgValve_nosts_11_23.sav')

['rf_gs_stacked_model_cabgValve_nosts_11_23.sav']

#### Loading saved `rf_gs_stacked_model`

In [203]:
#rf_gs_stacked_model = joblib.load('rf_gs_stacked_model_11_17.sav')

- analyzing results

In [288]:
rf_gs_stacked_model.best_params_

{'max_depth': None,
 'max_features': 0.1,
 'min_samples_split': 10,
 'n_estimators': 250}

In [289]:
gs_results = pd.DataFrame(rf_gs_stacked_model.cv_results_)

In [290]:
gs_results = gs_results.sort_values(by=['mean_test_roc_auc'],
                                           ascending=False)

In [291]:
display_cols = ['params',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']

In [292]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)

In [293]:
gs_results_summary.head()

,params,mean_test_roc_auc,mean_train_roc_auc,mean_test_f1,mean_test_precision,mean_test_recall
0,"{'max_depth': None, 'max_features': 0.1, 'min_samples_split': 10, 'n_estimators': 250}",0.67198,1.0,0.01538,0.19995,0.008
1,"{'max_depth': None, 'max_features': 0.1, 'min_samples_split': 5, 'n_estimators': 300}",0.66994,1.0,0.01538,0.19995,0.008
2,"{'max_depth': None, 'max_features': 0.1, 'min_samples_split': 10, 'n_estimators': 300}",0.66966,1.0,0.01538,0.19995,0.008
3,"{'max_depth': None, 'max_features': 0.1, 'min_samples_split': 5, 'n_estimators': 250}",0.66889,1.0,0.01538,0.19995,0.008
4,"{'max_depth': None, 'max_features': 0.1, 'min_samples_split': 10, 'n_estimators': 200}",0.66834,1.0,0.01538,0.19995,0.008


#### Key Takeaways
- better than `STS` alone, but results slightly below that of `LogisticRegression()`

### `Polynomial` Transformation on Stacked Model
- to see if adding interaction terms between `PREOP` models predicted probabilities and `POSTOP` features can improve results

- instantiating `Pipeline` that will include `Polynomial` transformation of `X_train_POSTOP_STACKED`

In [210]:
poly_lr_model_pipe = Pipeline(steps=[
    ('poly_features', PolynomialFeatures(degree=2, include_bias=False)),
    ('Classifier', LogisticRegression())
    ])

- defining a parameter grid for `GridSearchCV`

In [211]:
parameter_grid = {'Classifier__penalty': ['l1'],
                  'Classifier__C': [0.0001, 0.001, 0.01, 0.10, 0.50, 1.0, 10.0],
                  'Classifier__class_weight': ['balanced'],
                  'Classifier__solver': ['liblinear'],
                  'Classifier__random_state': [0]
                 }

- create `GridSearchCV` object

In [212]:
gs = GridSearchCV(poly_lr_model_pipe, # estimator
                  param_grid=parameter_grid, # param_grid
                  scoring=scoring_metrics, # loss functions
                  cv=5,
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  verbose=2,
                  return_train_score=True,
                  n_jobs=-1)

- fitting `GridSearchcV`

- `GridSearchCV` takes over 2 full hours to run

#### Saving `poly_log_reg_lasso`

In [213]:
#joblib.dump(poly_log_reg_lasso, 'poly_log_reg_lasso_model_11_17.sav')

#### Loading `poly_log_reg_lasso`

In [214]:
poly_log_reg_lasso = joblib.load('poly_log_reg_lasso_model_11_17.sav')

- examining results

In [215]:
poly_log_reg_lasso.best_params_

{'Classifier__C': 0.01,
 'Classifier__class_weight': 'balanced',
 'Classifier__penalty': 'l1',
 'Classifier__random_state': 0,
 'Classifier__solver': 'liblinear'}

In [216]:
gs_results = pd.DataFrame(poly_log_reg_lasso.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                      ascending=False)

In [217]:
display_cols = ['params',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']

In [218]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)

In [219]:
gs_results_summary

,params,mean_test_roc_auc,mean_train_roc_auc,mean_test_f1,mean_test_precision,mean_test_recall
0,"{'Classifier__C': 0.01, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'Cl...",0.69959,0.78409,0.06840,0.03647,0.54979
1,"{'Classifier__C': 0.001, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'C...",0.65922,0.66717,0.05326,0.02775,0.66316
2,"{'Classifier__C': 0.1, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'Cla...",0.61247,0.90305,0.06275,0.03429,0.36941
3,"{'Classifier__C': 0.5, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'Cla...",0.54727,0.94213,0.05680,0.03157,0.28345
4,"{'Classifier__C': 0.0001, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', '...",0.53768,0.54504,0.03347,0.01702,1.00000
5,"{'Classifier__C': 1.0, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'Cla...",0.52516,0.95176,0.05498,0.03078,0.25774
6,"{'Classifier__C': 10.0, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'Cl...",0.48404,0.96689,0.05166,0.02937,0.21479


#### Key Takeways
- `PolynomialTransformation` actually had worse results than non-transformed `LogisticRegression`
- Results were in-line with `STS` model

## CODE BELOW IS USEFUL FOR DIAGNOSTIC CHARTS ON MODEL SCORING

### Need to modify and adapt for code above given changes in variable names

In [220]:
break

SyntaxError: 'break' outside loop (<ipython-input-220-6aaf1f276005>, line 4)

# ---------------------------------------------------------------------------------------------

## Visualizing the Effect of `LogisticRegression()` Hyperparameter Values
- See Albon `11.13` pages 205-207
- range of values for `C` - inverse of regularization strength

In [ ]:
param_values = np.linspace(0.001, 0.05, 25)

- calculating `roc_auc` on training and test set using `param_range` for `C`

In [ ]:
train_scores, test_scores = validation_curve(LogisticRegression(penalty='l1',
                                                                class_weight='balanced',
                                                                random_state=0,
                                                                solver='liblinear'), #classifier
                                             X_train, #feature matrix
                                             y_train, #target vector
                                             param_name='C', #hyperparameter to examine,
                                             param_range=param_values, #range of hyperparameter values
                                             cv=5, #number of folds, default=`StratifiedKFold`
                                             scoring='roc_auc', #scoring metric
                                             n_jobs=-1, #use all computer cores
                                             verbose=5) 

- calculate `mean` and `standard deviation` for `training` set scores

In [ ]:
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

- calculate `mean` and `standard deviation` for `test` set scores

In [ ]:
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

- Plotting

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('fivethirtyeight')

# plot mean accuracy scores for training and test sets
plt.plot(param_values,
         train_mean,
         label='Training Score',
         color='midnightblue')

plt.plot(param_values,
         test_mean,
         label='Cross Validation Score',
         color='darkviolet')

# plot accuracy bands for training and test sets
plt.fill_between(param_values, 
                 train_mean - train_std,
                 train_mean + train_std,
                 color='lightsteelblue')

plt.fill_between(param_values,
                 test_mean - test_std,
                 test_mean + test_std,
                 color='thistle')

# create plot
plt.title('Validation Curve for Logistic Regression with l1 Regularization')
plt.xlabel('C')
plt.ylabel('AUCROC')
plt.legend(loc='lower right')
plt.show()       

In [ ]:
validation_summary = pd.DataFrame({'C': param_values,
                                   'mean_test_roc_auc': test_mean,
                                   'mean_train_roc_auc': train_mean})

In [ ]:
validation_summary[validation_summary['mean_test_roc_auc'] == np.max(validation_summary['mean_test_roc_auc'])]

#### How does this tuned model compare to the baseline `STS` Model

In [ ]:
round(roc_auc_score(y_train, predstro_train), 5)

#### Key Takeaways
- Fine tuning of hyperparmeter `C` results in `C=0.0275`
- Results in-line (but still lower) than `STS` Model

## Precision-Recall Curve for Tuned Logistic Regression Model
- `Hands On Machine Learning with Scikit-Learn & Tensorflow` by Aurelien Geron pages 83-93
#### How `cross_val_predict` works:
- `cross_val_predict` returns for each element in the input, the prediction that was obtained for that element when it was in the `test` set

- generating vector of probabilities of stroke for cross validated training set

In [ ]:
lr_predict_probs = cross_val_predict(LogisticRegression(penalty='l1',
                                                        C=0.0275,
                                                        class_weight='balanced',
                                                        random_state=0,
                                                        solver='liblinear'),
                                     X_train,
                                     y_train,
                                     cv=5,
                                     method='predict_proba',
                                     n_jobs=-1,
                                     verbose=5)

- using `lr_predict_probs` can compute precision and recall for all possible thresholds using the `precision_recall_curve` function
- returns an `numpy.ndarray` with `shape` `(n, 2)` as it returns a probability per class `(0, 1)`
- since we are interested in the `1` class, need to carve our the second column of the `numpy.ndarray`
- add `[:,1]`

In [ ]:
type(lr_predict_probs)

In [ ]:
lr_predict_probs.shape

In [ ]:
lr_predict_probs[0:5]

In [ ]:
lr_predict_probs[0:5, 1]

#### Generating `precision_recall_curve`

In [ ]:
precisions, recalls, thresholds = precision_recall_curve(y_train,
                                                         lr_predict_probs[:, 1])

- `precision` values such that element `i` is the the `precision` of predictions with `score>=threholds[i]` and the last element is `1`

In [ ]:
precisions[0:5] # first 5 elements of the numpy array

In [ ]:
precisions[-5:] # last 5 elements of the numpy array

- decreasing `recall` values such that element `i` is the the `precision` of predictions with `score>=threholds[i]` and the last element is `0`

In [ ]:
recalls[0:5]

In [ ]:
recalls[-5:]

- increasing `thresholds` on the decision function used to compute `precision` and `recall`

In [ ]:
thresholds[0:5]

In [ ]:
thresholds[-5:]

### Plotting `precision` and `recall` as functions of `threshold`

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], 'b--', label='Precision')
    plt.plot(thresholds, recalls[:-1], 'g-', label='Recall')
    plt.title('Precision and Recall versus Threshold')
    plt.xlabel('Threshold')
    plt.legend(loc='upper right')
    plt.ylim([0, 1])

### Precision and Recall versus Threshold for Logistic Regression Model

In [ ]:
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

- helper function

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx], idx

- examining specific `threshold`, `Recall` and `Precision` trade-offs
- suppose you wanted a `recall` of `0.80`, what `threshold` will you need and what will the `precision` be?

In [ ]:
find_nearest(recalls, 0.80)

In [ ]:
print('{0:15} {1:5f}'.format('Recall:', recalls[14806]))
print('{0:15} {1:5f}'.format('Precision:', precisions[14806]))
print('{0:15} {1:5f}'.format('Threshold:', thresholds[14806]))

- see `Geron` page 90 for additional details

### Precision and Recall versus Threshold for STS Model

In [ ]:
sts_precisions, sts_recalls, sts_thresholds = precision_recall_curve(y_train,
                                                                     predstro_train)

In [ ]:
plot_precision_recall_vs_threshold(sts_precisions, sts_recalls, sts_thresholds)
plt.show()

## Precision-Recall Curves
- `Introduction to Machine Learning with Python` by Andreas C. Muller and Sarah Guido pages 289-296
- the closer a `precision-recall curve` stays to the upper-right corner, the better classifier

In [ ]:
def plot_precision_recall_curve(precisions, recalls):
    plt.plot(precisions,
             recalls,
             color='blue',
             label='Precision-Recall Curve')
    plt.title('Precision-Recall Curve')
    plt.ylabel('Recall')
    plt.xlabel('Precision')
    plt.legend(loc='upper right')

### `Precision-Recall Curve` for `LogisticRegression` Model

In [ ]:
plot_precision_recall_curve(precisions, recalls)
plt.show()

#### Area Under the `Precision-Recall Curve` for `LogisticRegression` Model

In [ ]:
round(auc(recalls, precisions), 5)

### `Precision-Recall Curve` for `STS` Model

In [ ]:
plot_precision_recall_curve(sts_precisions, sts_recalls)
plt.show()

#### Area Under the `Precision-Recall Curve` for `STS` Model

In [ ]:
round(auc(sts_recalls, sts_precisions), 5)

#### Key Takeaways
- `STS` model is superior to `LogisticRegression` model looking at `Precision-Recall Curves` and `PR AUC`

### Comparing `Logistic Regression` and `STS` Model `Precision-Recall` Curves

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('fivethirtyeight')

# plotting `LogisticRegression` Precision-Recall Curve
plt.plot(precisions,
         recalls,
         'b--',
         label='Logistic Regression Model')

# plotting `STS Model` Precision-Recall Curve
plt.plot(sts_precisions,
         sts_recalls,
         'g-',
         label='STS Model')

plt.title('Logistic Regression, STS Model Precision-Recall Curves')
plt.ylabel('Recall')
plt.xlabel('Precision')
plt.legend(loc='upper right')
plt.xlim([0, 0.10]) # cut off values > 0.10 because they were essentially zero
plt.show()

#### Key Takeaways
- plot clearly shows `STS` model outperperformance over `LogisticRegression`
- can use basic code here to plot `Precision-Recall Curves` from other models on the same plot for comparison

## Receiver Operating Characteristic (`ROC`) Curves
- Albon `11.5` pages 189-192
- a classifier that predicts every observation correctly would look like the solid `gray` line in the plotted `ROC` curves below
- the solid `gray` line goes straight up to the top immediately
- a classifier that predicts at random will appear as the diagonal line
- the better the model, the closer it is to the solid `gray` line
- the `ROC` curve represents the respective `TPR` and `FPR` for every probability threshold

- creating `true` and `false` probabilities

In [ ]:
false_positive_rate, true_positive_rate, threshold = roc_curve(y_train,
                                                               lr_predict_probs[:, 1])

- `plot_roc_curve` function

In [ ]:
def plot_roc_curve(fpr, tpr):
    plt.title('Receiver Operating Curve (ROC)')
    plt.plot(fpr, tpr)
    plt.plot([0, 1], ls='--')
    plt.plot([0, 0],
             [1, 0],
             c='0.7'),
    plt.plot([1, 1], c='0.7')
    plt.ylabel('True Positive Rate (Recall)')
    plt.xlabel('False Positive Rate')

#### `LogisiticRegression` Model `ROC` Curve

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('fivethirtyeight')
plot_roc_curve(false_positive_rate, true_positive_rate)
plt.show()

In [ ]:
round(roc_auc_score(y_train, lr_predict_probs[:, 1]), 5)

- suppose you want `recall` in a certain range - what is the `false positive rate` you would have to accept?

In [ ]:
lr_roc_curve = pd.DataFrame({'Recall': true_positive_rate,
                             'FPR': false_positive_rate})

- assume you want `recall` around `0.80`

In [ ]:
lr_roc_curve[lr_roc_curve['Recall'].between(0.799, 0.801)]

- could have also used the `find_nearest` helper function above

In [ ]:
lr_roc_curve[lr_roc_curve['Recall'] == (find_nearest(lr_roc_curve['Recall'], 0.80)[0])]

- suppose you want to know `recall` and `FPR` for a given threshold

In [ ]:
np.where(threshold == 0.10)[0]

In [ ]:
print('{0:15} {1:5f}'.format('Threshold:', threshold[11]))
print('{0:15} {1:5f}'.format('Recall:', true_positive_rate[11]))
print('{0:15} {1:5f}'.format('FPR:', false_positive_rate[11]))

#### `STS` Model `ROC` Curve

In [ ]:
sts_fpr, sts_tpr, threshold = roc_curve(y_train,
                                        predstro_train)

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('fivethirtyeight')
plot_roc_curve(sts_fpr, sts_tpr)
plt.show()

In [ ]:
round(roc_auc_score(y_train, predstro_train), 5)

#### Key Takeaways
- `STS` Model slightly outperforms `LogisticRegression`